# SORA - Stellar Occultation Reduction and Analysis

## _prediction()_ and PredictionTable Object Class

The prediction module within SORA was created to predict stellar occultation, organize the prediction and plot the occultation maps. The documentation at <font color=blue>add link</font> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the prediction module. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

### Index

1. [Predicting stellar occultation](#section_1)
2. [Setting and/or modifying parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_4)
5. [MPC observatories](#section_5)

<a id="section_1"></a>
### 1. Predicting stellar occultation

To predict a stellar occultation, the function _prediction()_ must be used. It will need an EphemKernel Object and the time interval for the search. The function will generate the ephemeris within the given interval (with a default  step of 60 seconds between positions, that may be changed by the user) and download the star coordinates from Vizier. For every star, it checks the closest ephemeris. An occultation is identified if the distance is smaller than the radius of the Earth plus the radius of the object (given in Ephem) plus the error of the ephemeris (multiplied by a sigma factor given by the user). In the next step, the coordinate of the star is propagated to date using proper motion and parallax. It results in the occultation parameters, such as the Closest Approach distance, the Position Angle, Velocity, etc.

The function returns a PredictionTable with all the information.

**All functions have their Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

In [1]:
from sora.prediction import prediction

In [2]:
prediction?

Signature:
prediction(
    ephem,
    time_beg,
    time_end,
    mag_lim=None,
    step=60,
    divs=1,
    sigma=1,
    log=True,
)
Docstring:
Predicts stellar occultations

Parameters:
    ephem (Ephem): Ephemeris. It must be an Ephemeris object.
    time_beg (str,Time): Initial time for prediction
    time_beg (str,Time): Final time for prediction
    mag_lim (int,float): Faintest Gmag for search
    step (int, float): step, in seconds, of ephem times for search
    divs (int): number of regions the ephemeris will be splitted
        for better search of occultations
    sigma (int,float): ephemeris error sigma for search off-Earth.
    log (bool): To show what is being done at the moment.

Return:
    predict (PredictionTable): PredictionTable with the occultation params for each event
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      function


**Importing and defining the Ephemeris to be used.**

In [3]:
from sora.ephem import EphemKernel
eph_chariklo = EphemKernel(name='Chariklo', spkid='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'])

In [4]:
pred = prediction(eph_chariklo,time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', mag_lim=18, divs=3)

Ephemeris was split in 3 parts for better search of stars

Searching occultations in part 1/3
Generating Ephemeris between 2017-06-20 00:00:00.000 and 2017-06-23 15:59:00.000 ...
    14 Gaia-DR2 stars downloaded
Identifying occultations ...

Searching occultations in part 2/3
Generating Ephemeris between 2017-06-23 16:00:00.000 and 2017-06-27 07:59:00.000 ...
    12 Gaia-DR2 stars downloaded
Identifying occultations ...

Searching occultations in part 3/3
Generating Ephemeris between 2017-06-27 08:00:00.000 and 2017-06-30 23:59:00.000 ...
    18 Gaia-DR2 stars downloaded
Identifying occultations ...
8 occultations found


<a id="section_2"></a>
### 2. The PredictionTable

The resulting variable of the prediction function is a PredictionTable Object. The PredictionTable Class was created to have all the information about the occultations predicted, plot occultation maps and save the table in other formats.

**The PredictionTable is a class that inherits all methods from Astropy.table.Table. All the functions present in Astropy Table should work for PredictionTable.**

More information about it at https://docs.astropy.org/en/stable/table/

**if the user asks to print the prediction, a summary of the table is shown.**

In [5]:
print(pred)

         Epoch             ICRS Star Coord at Epoch    ...  GAIA-DR2 Source ID
                                                       ...                    
----------------------- ------------------------------ ... -------------------
2017-06-21 09:57:43.550 18 55 36.17452 -31 31 19.03240 ... 6760228702284187264
2017-06-22 02:58:36.900 18 55 26.31662 -31 31 20.38513 ... 6760228839723992320
2017-06-22 21:18:48.250 18 55 15.65250 -31 31 21.67051 ... 6760223758801661440
2017-06-23 21:34:37.350 18 55 01.48124 -31 31 22.44183 ... 6760223513963694208
2017-06-24 10:13:39.750 18 54 54.06758 -31 31 22.36730 ... 6760226503261782656
2017-06-25 10:55:09.450 18 54 39.55295 -31 31 21.69987 ... 6760225163236852864
2017-06-26 08:14:35.350 18 54 26.97506 -31 31 20.51222 ... 6760226060885482624
2017-06-26 20:46:07.050 18 54 19.56972 -31 31 20.43304 ... 6760225712991422208


**to print the complete table, we must use:**

In [6]:
pred.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T  GAIA-DR2 Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg                     
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- -------------------
2017-06-21 09:57:43.550 18 55 36.17452 -31 31 19.03240 18 55 36.17498 -31 31 19.60516  0.573 179.41 -21.84  14.663 15.268 15.364  225 00:56   128   165 6760228702284187264
2017-06-22 02:58:36.900 18 55 26.31662 -31 31 20.38513 18 55 26.31684 -31 31 20.74213  0.357 179.55 -21.92  14.661 17.949 18.049  329 00:53   138   166 6760228839723992320
2017-06-22 21:18:48.250 18 55 15.65250 -31 31 21.67051 18 55 15.65248 -31 31 21.62190  0.049 359.72 -22.00  14.659 14.238 14.341   53 00:50 

*M-G-T stands for Moon-Geocenter-Target, the on-sky angle between the star and the moon*

*S-G-T stands for Sun-Geocenter-Target, the on-sky angle between the star and the sun*

**The PredictionTable can be exported to PRAIA format file**

In this case, some information, such as the Gaia-DR2 Source ID, S-G-T and M-G-T, are lost.

In [7]:
pred.to_praia?

Signature: pred.to_praia(filename)
Docstring:
Write PredictionTable to PRAIA format.

INPUT:
    filename(str): name of the file to save table
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      method


In [8]:
pred.to_praia('output/Chariklo_occs.table')
# Check your folder named output

**The PredictionTable can also generate the files in the Occult Watcher feed format**

These are two files, named 'LOG.dat' and 'tableOccult_update.txt'. The occult watcher designation for the object must be informed. For instance, for the satellite Himalia, the designation is "P5M06". If mode='append' the prediction will be appended to an existing file. If mode='restart', the files will overwritten.

In [9]:
pred.to_ow?

Signature: pred.to_ow(ow_des, mode='append')
Docstring:
Write PredictionTable to OccultWatcher feeder update files.
Tables will be saved in two files: "tableOccult_update.txt" and "LOG.dat"

INPUT:
    ow_des (str): Occult Watcher designation for the object.
    mode (str): 'append' to append table to already existing file, default
                'restart' to overwrite existing file.
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      method


In [10]:
pred.to_ow('1997CU26')

**Using the functions from Astropy Table, the data can be exported to many other formats using the funtion 'write'**

The formats allowed can be seen in https://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers. This includes latex and csv.

In [11]:
pred.write('output/prediction.tex')
# Check your folder named output

<a id="section_3"></a>
### 3. Using PredictionTable

The PredictionTable was not created to be instantiated directly by the user. It was designed to be an insteresting output object from the prediction function. However, a PredictionTable can be instantiated from a PRAIA occultation table. For this, a PredictionTable method must be called directly as shown below.

In [12]:
from sora.prediction import PredictionTable

In [13]:
PredictionTable.from_praia?

Signature: PredictionTable.from_praia(filename, name, **kwargs)
Docstring:
Create a PredictionTable Table reading from a PRAIA table

INPUT:
    filename (str): path to the PRAIA table file.
    name (str): Name of the Object of the prediction.
    radius (int,float): Object radius, in km. (not required)
        If not given it's searched in online database.
        If not found online, the default is set to zero.

OUTPUT:
    A PredictionTable
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      method


In [14]:
pred_1 = PredictionTable.from_praia(filename='output/Chariklo_occs.table', name='Chariklo')

In [15]:
pred_1.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T GAIA-DR2 Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg                    
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- ------------------
2017-06-21 09:57:43.500 18 55 36.17450 -31 31 19.03240 18 55 36.17500 -31 31 19.60520  0.573 179.41 -21.84  14.660 15.304 15.400  225 00:56   128   165                   
2017-06-22 02:58:36.900 18 55 26.31660 -31 31 20.38510 18 55 26.31680 -31 31 20.74210  0.357 179.55 -21.92  14.660 17.900 18.000  329 00:53   138   166                   
2017-06-22 21:18:48.200 18 55 15.65250 -31 31 21.67050 18 55 15.65250 -31 31 21.62190  0.049 359.72 -22.00  14.660 14.197 14.300   53 00:50   149

To remove some occultations the User can use two functions. The first is the _PredictionTable.remove_occ()_.

In [16]:
pred_1.remove_occ?

Signature: pred_1.remove_occ(date)
Docstring:
Remove stellar occultations from table

INPUT:
    date (str,list): Date or list of dates of the occultation to be removed.
        The dates mut be as shown in the 'Epoch' column. If the date is not
        complete, the function will select all occultations that matches
        the given string. For instance, date='2020-06' will remove all
        occultations from the month of June 2020.
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      method


In [17]:
pred_1.remove_occ('2017-06-21 09:57')

In [18]:
pred_1.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T GAIA-DR2 Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg                    
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- ------------------
2017-06-22 02:58:36.900 18 55 26.31660 -31 31 20.38510 18 55 26.31680 -31 31 20.74210  0.357 179.55 -21.92  14.660 17.900 18.000  329 00:53   138   166                   
2017-06-22 21:18:48.200 18 55 15.65250 -31 31 21.67050 18 55 15.65250 -31 31 21.62190  0.049 359.72 -22.00  14.660 14.197 14.300   53 00:50   149   166                   
2017-06-23 21:34:37.300 18 55 01.48120 -31 31 22.44180 18 55 01.48120 -31 31 22.22700  0.215 359.91 -22.10  14.660 17.792 17.900   48 00:46   162

The second function is the _PredictionTable.keep_from_selected_images()_ this function will check the names in the saved images and eliminate the occultations without the map. This functions allows that after the user deleted the maps of the unwanted occultation this function will eliminate the respective rows in the _PredictionTable_.  

In [19]:
pred_1.keep_from_selected_images?

Signature: pred_1.keep_from_selected_images(path='.')
Docstring:
Keeps predictions which images were not deleted in given path
This function uses the name of the images to identify predictions.
The name must be the automatic one generated by plot_occ_map().
The format of the image is not relevant.

INPUT:
    path (str): path where images are located
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      method


<a id="section_4"></a>
### 4. Plotting occultation maps

SORA is also able to generate an occultation map. The only required parameters are the occultation parameters. The function also has many inputs to configure the plot.

The first time the function is called, cartopy will download some data referring to the features presented in the map, such as the country and state division, lakes, rivers, etc.

In [20]:
from sora.prediction import plot_occ_map

In [21]:
plot_occ_map?

Signature:
plot_occ_map(
    name,
    radius,
    coord,
    time,
    ca,
    pa,
    vel,
    dist,
    mag=0,
    longi=0,
    **kwargs,
)
Docstring:
Plots map of the occultation

Parameters:
    Required params:
    name (str): Name of the object
    radius (int, float): radius of the object (km)
    coord (str, SkyCoord): Coordinate of the star
        ("hh mm ss.sss dd mm ss.sss" or "hh.hhhhhhhh dd.dddddddd")
    time (str, Time): Instant of Closest Approach (iso or isot format)
    ca (int, float): Closest Approach Distance (arcsec)
    pa (int, float): Position Angle at C/A (deg)
    vel (int, vel): Velocity of the event (km/s)
    dist (int, float): Object distance at C/A (AU)

    Not required params (only printed in label):
    mag (int,float): Mag* = Normalized magnitude to vel=20km/s
    longi (int,float): East longitude of sub-planet point, deg, positive towards East

    Map configuration:
    nameimg (str): Change the name of the imaged saved.
    path (str): Path to a

**To make it easier to plot the predictions, the** _PredictionTable.plot_occ_map()_ **function can be called which automatically fills the required parameters.**

To plot the map for all the predictions


In [22]:
pred.plot_occ_map(path='output/')

Chariklo_2017-06-21T09:57:43.550.png generated
Chariklo_2017-06-22T02:58:36.900.png generated
Chariklo_2017-06-22T21:18:48.250.png generated
Chariklo_2017-06-23T21:34:37.350.png generated
Chariklo_2017-06-24T10:13:39.750.png generated
Chariklo_2017-06-25T10:55:09.450.png generated
Chariklo_2017-06-26T08:14:35.350.png generated
Chariklo_2017-06-26T20:46:07.050.png generated


To plot the map for only one prediction, just give a item to the PredictionTable object.

In [23]:
pred[0].plot_occ_map(path='output/')

Chariklo_2017-06-21T09:57:43.550.png generated


**The PredictionTable can also be plotted by giving the date of the occultation. All the occultations that matches the date will be plotted.**

The date can be as constrained as the user wants, and must match the text that appears in the 'Epoch' column.

In [24]:
pred['2017-06-26'].plot_occ_map()

Chariklo_2017-06-26T08:14:35.350.png generated
Chariklo_2017-06-26T20:46:07.050.png generated


<a id="section_5"></a>
### 5. Occultation parameters

Finally, a function is implemented in the prediction module which calculates the occultation parameters. For this function, it must be passed a Star objetct, an Ephemeris object (it can be any of the Ephemeris classes) and a time. The time does not need to be precise, but it must be close within 10 minutes from the Closest Approach.

In [25]:
from sora.prediction import occ_params

In [26]:
occ_params?

Signature: occ_params(star, ephem, time)
Docstring:
Calculates the parameters of the occultation, as instant, CA, PA.

Parameters:
    star (Star): The coordinate of the star in the same frame as the ephemeris.
        It must be a Star object.
    ephem (Ephem): Ephemeris. It must be an Ephemeris object.

Return:
    instant of CA (Time): Instant of Closest Approach
    CA (arcsec): Distance of Closest Approach
    PA (deg): Position Angle at Closest Approach
    vel (km/s): Velocity of the occultation
    dist (AU): the object distance.
File:      ~/.local/lib/python3.7/site-packages/sora/prediction.py
Type:      function


In [27]:
from sora.star import Star
s = Star(code='6760225712991422208', log=False)

In [28]:
tca, ca, pa, vel, dist = occ_params(s, eph_chariklo, time='2017-06-26 20:40')
print('Time of the CA:   {} UTC'.format(tca))
print('Closest Approuch: {:.3f}'.format(ca))
print('Position Angle:   {:.3f}'.format(pa))
print('Shadow velocity:  {:.3f}'.format(vel))
print('Object distance:  {:.5f}'.format(dist))

Time of the CA:   2017-06-26 20:46:07.050 UTC
Closest Approuch: 0.198 arcsec
Position Angle:   0.529 deg
Shadow velocity:  -22.332 km / s
Object distance:  14.65230 AU


**This Jupyter-Notebook was designed as a tutorial for how to work with the** _prediction()_ **functions and PredictionTable Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

# The End